In [ ]:
from selenium import webdriver
import openpyxl
import pytesseract
from PIL import Image
import time
import csv
from bs4 import BeautifulSoup

# Set the Tesseract OCR executable path
pytesseract.pytesseract.tesseract_cmd = 'C:/Program Files/Tesseract-OCR/tesseract.exe'

# Define the base URL
base_url = "https://old.icegate.gov.in/EnqMod/searchIecCodeAction"

# Initialize the WebDriver
driver = webdriver.Chrome(executable_path="C:/Users/hites/Downloads/chromedriver_win32/chromedriver.exe")
driver.maximize_window()
driver.implicitly_wait(10)

# Load the base URL
driver.get(base_url)

# Initialize a list to store IE Codes
ie_code = []

# Function to read IE Codes from an Excel file
def read_csv():
    wb_obj = openpyxl.load_workbook('test.xlsx')
    ws = wb_obj.active

    for index, col in enumerate(ws['A']):
        if index != 0:
            ie_code.append(col.value)

# Class for web scraping
class Python_Scrap:
    def __init__(self):
        self.input_Ie = driver.find_element("id", 'searchIECode')
        self.input_captcha = driver.find_element("id", 'captchaResp')
        self.captcha = driver.find_element('xpath', "//dt[@id='login-box-name']//img")
        self.column_list = ['IE Code', 'Name', 'Address', 'IEC Status', 'PAN']

    def input_element(self, ie_number):
        self.input_Ie.send_keys(ie_number)
        self.captcha.screenshot('1.png')
        image_url = Image.open('1.png')
        text = pytesseract.image_to_string(image_url).strip()
        self.input_captcha.send_keys(text)
        time.sleep(3)
        driver.find_element_by_xpath("//*[contains(text(), 'IE Code@ICEGATE')]").click()
        table_to_extract = driver.find_elements('xpath', "//*[contains(text(), 'IEC Details')]")

        if not table_to_extract:
            time.sleep(2)
            self.input_element(ie_number)
        else:
            time.sleep(2)
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            table = soup.find('table', id="pagetable")
            if table:
                new_row = []

                for row in table.find_all('tr'):
                    for column in row.find_all('td'):
                        curr_column = column.get_text().strip()
                        if curr_column and curr_column not in self.column_list:
                            new_row.append(curr_column)

                with open("data.csv", 'a', newline='') as csvfile:
                    wr = csv.writer(csvfile)
                    wr.writerow(new_row)

                driver.back()
                time.sleep(2)

# Read IE Codes from Excel
read_csv()

# Iterate through IE Codes and scrape data
if __name__ == '__main__':
    for i in ie_code:
        ps = Python_Scrap()
        ps.input_element(i)

# Close the WebDriver
driver.close()